In [13]:
from pathlib import Path
from typing import List

from os import path
import numpy as np
from skimage import io,util,filters
from skimage.transform import EuclideanTransform, warp
from scipy import ndimage

from pystackreg import StackReg
# from twophotonUtils import parse_unregistered_channels
# from twophotonUtils import find_most_likely_z_slice_using_CC, z_translate_and_pad
# from imageLoadingWidgets import LoadTimepointForInspection

from os import path
from glob import glob
from re import findall
from tqdm import tqdm

import stackview

# Extract the first ome.tiff file from every subfolder, load, then separate the two channels
def sort_by_slice(filename):
    z = findall('_(\d+).ome.tif',filename)[0]
    return int(z)


# 0. Get file paths

Step one: Load all the OME-TIFFs and re-save as multipage TIFFs for each time point in region
Will perform a StackReg on each stack just in case there is small movement.


In [2]:
dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/'

subfolders = glob(path.join(dirname,'Left paw */*/ZSeries*[!zoom1]/'))

header_ome_h2b = []
header_ome_fucci = []

for d in subfolders:
    
    ome_tifs = glob(path.join(d,'*.ome.tif'))
    ome_tifs = sorted(ome_tifs) # Sort by channel #
    ome_tifs = sorted(ome_tifs, key = sort_by_slice) # Sort by slice #
    if len(ome_tifs) < 40:
        print(f'Skipping {d}')
    else:
        if len(findall('1020nm',path.split(path.split(d)[0])[1])) == 0:
            header_ome_h2b.append(ome_tifs[0])
        else:
            header_ome_fucci.append(ome_tifs[0])

print(f'Found {len(header_ome_h2b)} B/G stacks and {len(header_ome_fucci)} R/R_shg stacks')

Found 6 B/G stacks and 0 R/R_shg stacks


In [11]:
import warnings
warnings.filterwarnings("ignore")

names = ['R','G','B']
registerZ = False; reference = 'B'

for header_ome in tqdm(header_ome_h2b):
    d = path.split(path.dirname(header_ome))[0]
    if path.exists(path.join(d,'R_reg.tif')):
        continue
    
    # Load ome-tif
    print(f'Loading {d}')
    stack = io.imread(header_ome,is_ome=True)
    
    if stack.ndim > 3:
        images = [im for im in stack]
    else:
        images = [stack]

    if registerZ and reference is not None:
        # Use StackReg
        print(f'Registering {d}')
        sr = StackReg(StackReg.TRANSLATION) # There should only be slight sliding motion within a single stack
        T = sr.register_stack(B,reference='previous',n_frames=20,axis=0) #Obtain the transformation matrices
        for i,im in enumerate(images):
            images[i] = sr.transform_stack(im,tmats=T) # Apply to all channels

    for i,im in enumerate(images):
        output_path = path.join( d,f'{names[i]}_reg.tif')
        io.imsave(output_path,util.img_as_uint(im/im.max()),check_contrast=False)
    
    print(f'Saved with {output_path}')

  0%|                                                             | 0/6 [00:00<?, ?it/s]<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no dat

Loading /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/Left paw R2/1. Day 11


<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised

Saved with /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/Left paw R2/1. Day 11/B_reg.tif
Loading /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/Left paw R1/1. Day 11


<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading CreateTime raised ValueError('no datetime before year 1 (julianday=0)')
<tifffile.read_uic_tag> reading LastSavedTime raised

Saved with /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/10-03-2025 Cdh5-RFP blood vessels/Male 6121/Left paw R1/1. Day 11/B_reg.tif


In [17]:
stackview.switch({names[i]:images[i] for i in range(len(names))},
                colormap=['pure_red','pure_green','pure_blue'],
                toggleable=True,
                zoom_factor=0.5)
